In [ ]:
import pickle
import warnings
from datetime import datetime, timedelta
import random
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap
import pacmap
import Levenshtein

from manage import jsonAttempts2data, jsonExercises2data
from code2aes import Code2Aes
from aes2vec import learnModel, inferVectors, read_corpus, data2cor


C:\Users\Stagiaire\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In this notebook, we will compare the impact of PCA, t-SNE, and PaCMAP on dimensionality reduction.
To do this, we will compute the similarity between all points in the original embedding, then compute the similarity in the reduced embeddings and calculate the difference. A lower score means a better representation

First, let's import some data (trajectories...)

In [2]:
# # Importations des données
# NC1014 = jsonAttempts2data('Datasets/NewCaledonia_1014.json')
# NCExercises = jsonExercises2data('Datasets/NewCaledonia_exercises.json')
# NC5690 = jsonAttempts2data('Datasets/NewCaledonia_5690.json')

In [ ]:
# Import the attemps' embedding and data
with open("Datasets/Embedding/embedding_dublin.json", "r") as f:
    embedding_data = json.load(f)

with open("Datasets/Embedding/embedding_correction_dublin.json", "r") as f:
    embedding_data_correction = json.load(f)

with open("Datasets/Raw/DBExercises.json", "r") as f:
    DBExercises = json.load(f)

chemin = "Datasets/Raw/Dublin_42487.json"
with open(chemin, 'r') as fichier:
    dublin_data = json.load(fichier)

In [4]:
# Load data trajectories

with open('Datasets\data_visualisation_dublin_dublin.pkl', 'rb') as fichier:
    data_visualisation = pickle.load(fichier)
trajec_emb = data_visualisation[0]

In [6]:
# similarity measure 
def cos2(x,y):
    """
    Similarity measure
    Return : float between 0 and 1
    A value of 1 indicates that x and y are similar, while a value of 0 indicates that they are not.
    """
    prod = np.dot(x,y)
    norm1 =  np.linalg.norm(x)
    norm2 =  np.linalg.norm(y)
    cos2 = prod / (norm1 * norm2)
    return cos2 ** 2

In [7]:
def distance_embedding(trajec):
    """
    Compute the similarity of every point with every other point.
    Return: Dictionary where keys are exercises, and values are lists of lists of similarities.
    The i-th list contains the similarity of point i with every other point
    """
    score_emb = {}
    for exercise in trajec:
        score_emb[exercise] = []
        embeddings = trajec[exercise]
        for i in range(len(embeddings)-1):
            embbeding_compared = embeddings[i]
            compare = []
            for j in range(i+1,len(embeddings)):
                embbeding_compare = embeddings[j]
                score = cos2(embbeding_compared,embbeding_compare)
                compare.append(score)
            score_emb[exercise].append(compare)
    return score_emb

Let's get the reduced dimension of embedding with t-sne, pca and pacmap

In [8]:
def get_reduced_data(trajec_emb):
    warnings.filterwarnings('ignore')
    trajectory_reduced = {"t_sne" : {}, "PCA" : {}, "pacmap" : {}}
    for exo in tqdm(trajec_emb.keys()):
        list_emb = trajec_emb[exo]
        data_array = np.array(list_emb)
        n_samples = data_array.shape[0]
        perplexity = min(n_samples - 1, 30)
        # Reduced data with TSNE
        embedding_TSNE = TSNE(n_components=2, perplexity=perplexity, random_state=42)
        X_TSNE = embedding_TSNE.fit_transform(data_array)
        trajectory_reduced["t_sne"][exo] = X_TSNE
        # Reduced data with PCA
        embedding_PCA = PCA(n_components=2, random_state=42)
        X_PCA = embedding_PCA.fit_transform(data_array)
        trajectory_reduced["PCA"][exo] = X_PCA
        # Reduced data with pacmap
        embedding_pacmap = pacmap.PaCMAP(n_components=2, n_neighbors=perplexity, random_state=42)
        X_pacmap = embedding_pacmap.fit_transform(data_array)
        trajectory_reduced["pacmap"][exo] = X_pacmap
    return trajectory_reduced

In [9]:
def score(trajectory_reduced):
    """
    Compute the score (cos²) between every points for each algo
    """
    score_algo = {}
    for algo in tqdm(trajectory_reduced):
        trajec = trajectory_reduced[algo]
        score_emb = distance_embedding(trajec)
        score_algo[algo] = score_emb
    return score_algo

In [10]:
def compare_visu(score_method, score_embbeding):
    score_compare = {}
    for algo in score_method:
        score_compare[algo] = {}
        for exercise in score_embbeding:
            score_emb = score_embbeding[exercise]
            score_algo = score_method[algo][exercise]
            score = 0
            for i in range(len(score_emb)):
                score_array = np.array(score_emb[i])
                score_algo_array = np.array(score_algo[i])
                score += sum(abs(score_array-score_algo_array))
            score_compare[algo][exercise] = score
    return score_compare

Let's compute some random embedding for the baseline

In [11]:
# m for min and M for max
m, M = 0,0
for exercise in trajec_emb:
        for i in range(len(trajec_emb[exercise])):
            m = min(m,min(trajec_emb[exercise][i]))
            M = max(M,max(trajec_emb[exercise][i]))

In [ ]:
trajec_less_2 = []
for exercise in trajec_emb:
    if len(trajec_emb[exercise]) == 2:
        trajec_less_2.append(exercise)

for exercise in trajec_less_2:
    del trajec_emb[exercise]

In [16]:
# Create random embedding
random_embedding = {}
for exercise in tqdm(trajec_emb):
    random_embedding[exercise] = {}
    random_embedding[exercise] = []
    for i in range(len(trajec_emb[exercise])):
        random_values = [random.uniform(m, M) for _ in range(100)]
        random_embedding[exercise].append(random_values)

  0%|          | 0/114 [00:00<?, ?it/s]

In [15]:
score_emb = distance_embedding(trajec_emb)
trajectory_reduced = get_reduced_data(trajec_emb)
score_algo = score(trajectory_reduced)
score_compare = compare_visu(score_algo, score_emb)

score_emb ok


  0%|          | 0/114 [00:00<?, ?it/s]

  File "C:\Users\Stagiaire\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(


trajectory_reduced ok


  0%|          | 0/3 [00:00<?, ?it/s]

score_algo ok


In [20]:
score_emb_random = distance_embedding(random_embedding)
trajectory_reduced_random = get_reduced_data(random_embedding)
score_algo = score(trajectory_reduced_random)
score_compare_random = compare_visu(score_algo, score_emb_random)

  0%|          | 0/114 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
print("Using real data, we get : ")
print("\n")
for algo in score_compare:
    score = score_compare[algo]
    mean = np.mean(list(score.values()))
    print(f"{algo} got a mean score of {mean}")

Using real data, we get : 


t_sne got a mean score of 42480.36378040209
PCA got a mean score of 39050.16639876436
pacmap got a mean score of 46024.04249140676


In [22]:
print("Using random data we get: ")
print("\n")
for algo in score_compare_random:
    score = score_compare_random[algo]
    mean = np.mean(list(score.values()))
    print(f"{algo} got a mean score of {mean}")

Using random data we get: 


t_sne got a mean score of 53562.3003233795
PCA got a mean score of 53570.41282718403
pacmap got a mean score of 53550.39569684557


Let's compare t-sne, pacmap and PCA using the source code

In [23]:
def euclidean_distance(x, y):
    return np.sqrt((x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2)

In [24]:
def closest_point(trajectory_reduced, algo1, algo2):
    """
    trajectory_reduced : dict
    algo1, algo2  : str
    return dict
    """
    closest_point_algo1 = {}
    closest_point_algo2 = {}

    for exo in tqdm(trajectory_reduced[algo1]):
        if exo not in closest_point_algo1:
            closest_point_algo2[exo] = {}
            closest_point_algo1[exo] = {}
        coord_algo2 = trajectory_reduced[algo2][exo]
        coord_algo1 = trajectory_reduced[algo1][exo]
        for i, coord_t in enumerate(coord_algo2):
            coord_p = coord_algo1[i]
            if i not in closest_point_algo1[exo]:
                closest_point_algo1[exo][i] = []
                closest_point_algo2[exo][i] = []
            for j in range(len(coord_algo2)):
                    coord_algo1_compare = coord_algo1[j]
                    coord_algo2_compare = coord_algo2[j]
                    dist_algo1 = euclidean_distance(coord_p,coord_algo1_compare)
                    dist_algo2 = euclidean_distance(coord_t,coord_algo2_compare)
                    closest_point_algo1[exo][i].append(dist_algo1)
                    closest_point_algo2[exo][i].append(dist_algo2)
    return closest_point_algo1, closest_point_algo2

In [25]:
def closet_point_lev(trajectory_reduced, data_visualisation):
    source_code = data_visualisation[4]
    closest_point_lev = {}
    for exo in tqdm(trajectory_reduced["PCA"]):
        if exo not in closest_point_lev:
            closest_point_lev[exo] = {}
        codes = source_code[exo]
        for i, code in enumerate(codes):
            if i not in closest_point_lev[exo]:
                closest_point_lev[exo][i] = []
                for j in range(len(codes)):
                    code_compare = codes[j]
                    dis_lev = Levenshtein.distance(code, code_compare)
                    closest_point_lev[exo][i].append(dis_lev)
    return closest_point_lev

In [26]:
def define_closest_point(closest_point):
    closest_point_index = {}
    for exo in closest_point:
        closest_point_index[exo] = {}
        for index_tentative in closest_point[exo]:
            list_distance = closest_point[exo][index_tentative]
            closet_point = [i for i in range(len(list_distance))]
            paired_lists = list(zip(list_distance, closet_point))
            paired_lists_sorted = sorted(paired_lists, key=lambda x: x[0])
            sorted_distances, sorted_closet_points = zip(*paired_lists_sorted)
            sorted_distances = list(sorted_distances)
            sorted_closet_points = list(sorted_closet_points)
            closest_point_index[exo][index_tentative] = sorted_closet_points
    return closest_point_index

In [27]:
def count_permutations(reference_list, target_list):
    index_map = {value: idx for idx, value in enumerate(reference_list)}
    
    visited = [False] * len(target_list)
    swaps = 0
    
    for i in range(len(target_list)):
        if visited[i] or index_map[target_list[i]] == i:
            # If already visited or in the correct position, skip
            continue
        
        # Start of a new cycle
        cycle_size = 0
        j = i
        
        while not visited[j]:
            visited[j] = True
            j = index_map[target_list[j]]
            cycle_size += 1
        
        # Each cycle of size n requires n-1 swaps
        if cycle_size > 0:
            swaps += (cycle_size - 1)
    
    return swaps

In [28]:
def permuation_algo(closest_point_pca, closest_point_tsne, closest_point_lev):
    permu_pca = {}
    permu_tsne = {}
    for exo in closest_point_lev:
        permu_tsne[exo] = 0
        permu_pca[exo] = 0
        for i in closest_point_pca[exo]:
            reference_list = closest_point_lev[exo][i]
            list_tsne = closest_point_tsne[exo][i]
            list_acp = closest_point_pca[exo][i]
            num_perm_pca = count_permutations(reference_list, list_acp)
            num_perm_tsne = count_permutations(reference_list, list_tsne)
            permu_tsne[exo] += num_perm_tsne
            permu_pca[exo] += num_perm_pca
    return permu_tsne, permu_pca

In [29]:
closest_point_pca, closest_point_tsne = closest_point(trajectory_reduced, "PCA", "t_sne")
closest_point_lev = closet_point_lev(trajectory_reduced, data_visualisation)

closest_point_pca = define_closest_point(closest_point_pca)
closest_point_tsne = define_closest_point(closest_point_tsne)
closest_point_lev = define_closest_point(closest_point_lev)


permu_tsne, permu_pca = permuation_algo(closest_point_pca, closest_point_tsne, closest_point_lev)

  0%|          | 0/114 [00:00<?, ?it/s]

  0%|          | 0/114 [00:00<?, ?it/s]

In [30]:
mean_value_t = np.mean(list(permu_tsne.values()))
mean_value_a = np.mean(list(permu_pca.values()))
print(f"On average, we got {int(mean_value_t)} permutations for tsne agaisnt {int(mean_value_a)} for pca")

On average, we got 215483 permutations for tsne agaisnt 215535 for pca


In [31]:
closest_point_pca, closest_point_pacmap = closest_point(trajectory_reduced, "PCA", "pacmap")

closest_point_pca = define_closest_point(closest_point_pca)
closest_point_pacmap = define_closest_point(closest_point_tsne)

permu_pacmap, permu_pca = permuation_algo(closest_point_pca, closest_point_pacmap, closest_point_lev)

  0%|          | 0/114 [00:00<?, ?it/s]

In [32]:
mean_value_t = np.mean(list(permu_pacmap.values()))
mean_value_a = np.mean(list(permu_pca.values()))
print(f"On average, we got {int(mean_value_t)} permutations for pacmap agaisnt {int(mean_value_a)} for pca")

On average, we got 216109 permutations for pacmap agaisnt 215535 for pca


In [33]:
closest_point_tsne, closest_point_pacmap = closest_point(trajectory_reduced, "t_sne", "pacmap")

closest_point_tsne = define_closest_point(closest_point_tsne)
closest_point_pacmap = define_closest_point(closest_point_tsne)

permu_pacmap, permu_tsne = permuation_algo(closest_point_tsne, closest_point_pacmap, closest_point_lev)

  0%|          | 0/114 [00:00<?, ?it/s]

In [34]:
mean_value_t = np.mean(list(permu_pacmap.values()))
mean_value_a = np.mean(list(permu_tsne.values()))
print(f"On average, we got {int(mean_value_t)} permutations for pacmap agaisnt {int(mean_value_a)} for tsne")

On average, we got 216109 permutations for pacmap agaisnt 215483 for tsne
